In [ ]:
#mounting drive
from google.colab import drive
drive.mount('/content/drive')

Importing libraries

In [ ]:
!pip install -q condacolab
import condacolab
condacolab.install()

In [ ]:
!curl -Lo conda_installer.py https://raw.githubusercontent.com/deepchem/deepchem/master/scripts/colab_install.py
#import conda_installer
#conda_installer.install()
!/root/miniconda/bin/conda info -e

In [ ]:
!pip install --pre deepchem
import deepchem as dc
dc.__version__

In [ ]:
!pip install mordred
!pip install boruta

In [ ]:
!wget -c https://repo.continuum.io/miniconda/Miniconda3-py37_4.8.3-Linux-x86_64.sh
!chmod +x Miniconda3-py37_4.8.3-Linux-x86_64.sh
!time bash ./Miniconda3-py37_4.8.3-Linux-x86_64.sh -b -f -p /usr/local
!time conda install -q -y -c conda-forge rdkit

import sys
sys.path.append('/usr/local/lib/python3.7/site-packages/')

In [ ]:
!pip install dgl

In [ ]:
!pip install dgllife
!pip install imblearn

In [ ]:
from sklearn.model_selection import StratifiedKFold
import imblearn
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import train_test_split
import deepchem as dc
import numpy as np
import pandas as pd
import pickle
import tensorflow as tf
from collections import Counter
import warnings
warnings.filterwarnings("ignore")
from statistics import mean
from sklearn.metrics import roc_curve, auc
import numpy as np
from deepchem.feat import MolGraphConvFeaturizer
from deepchem.feat.graph_data import GraphData
from deepchem.feat.mol_graphs import ConvMol
from deepchem.data import NumpyDataset
from mordred import Calculator, descriptors
from rdkit import Chem
from imblearn.over_sampling import SMOTE
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,roc_auc_score,cohen_kappa_score,f1_score,precision_score,recall_score
from collections import Counter
import sys
from sklearn.ensemble import RandomForestClassifier,ExtraTreesClassifier
import warnings
from boruta import BorutaPy
import joblib

#1. Remove invalid samples

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/ensemble/Data/input.csv')
print(len(data))

1.1 MolGraphConv

In [ ]:
def generate_dataset(data,feat):  
  smiles = data['smiles']
  featurizer = feat
  features = featurizer.featurize(smiles)  
  indices = [ i for i, data in enumerate(features) if type(data) is not GraphData] #indices which failed to featurize  
  return indices

In [ ]:
#input data
data = pd.read_csv('/content/drive/MyDrive/ensemble/Data/input.csv')
data = data
feat = dc.feat.MolGraphConvFeaturizer()
while True:# remove all error producing samples
  indices = generate_dataset(data,feat)
  if len(indices) > 0: #keep removing unless there are no error prone samples
    data.drop(index=indices,inplace=True)
  else:
    break
  print(len(data))

In [ ]:
data.to_csv('/content/drive/MyDrive/ensemble/Data/input_molgraphconv.csv',index=False)#After 1st level of filtering

1.2 ConvMol

In [ ]:
def generate_dataset(data,feat):  
  smiles = data['smiles']
  featurizer = feat
  features = featurizer.featurize(smiles)  
  indices = [ i for i, data in enumerate(features) if type(data) is not ConvMol]   
  return indices

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/ensemble/Data/input_molgraphconv.csv')
feat = dc.feat.ConvMolFeaturizer()
while True:# remove all error producing samples
  indices = generate_dataset(data,feat)
  if len(indices) > 0: #keep removing unless there are no error prone samples
    data.drop(index=indices,inplace=True)
  else:
    break
  print(len(data))

In [ ]:
data.to_csv('/content/drive/MyDrive/ensemble/Data/input_clean.csv',index=False)

#2. Extract Mordred Features

2.1 Apoptosis

In [ ]:
data1 = pd.read_csv('/content/drive/MyDrive/ensemble/Data/apoptosis.csv')
data2 = pd.read_csv('/content/drive/MyDrive/ensemble/Data/input_clean.csv')
data2.head()
print(len(data1),len(data2))

In [ ]:
data = pd.concat((data1,data2),axis=0)
print(len(data))

In [ ]:
#input
def extract_features(data): #extracting mordred features for apoptosis data
  calc = Calculator(descriptors)
  mols = [Chem.MolFromSmiles(smi) for smi in data["smiles"]]  
  df = calc.pandas(mols)  ## All features  
  df["status"] = data["status"].values   
  df.to_csv('/content/drive/MyDrive/ensemble/Data/apoptosis_input_mordred.csv',index=False)
  return df

In [ ]:
feat = extract_features(data)

2.2 Cell Proliferations

In [ ]:
data1 = pd.read_csv('/content/drive/MyDrive/ensemble/Data/cell_proliferation.csv')
data1 = data1[['name','smiles','status']]
data2 = pd.read_csv('/content/drive/MyDrive/ensemble/Data/input_clean.csv')
data2 = data2[['name','smiles']]
data2['status'] = [0]*len(data2) #to maintain consistency
print(len(data1),len(data2))

In [ ]:
data = pd.concat((data1,data2),axis=0)
print(len(data))

In [ ]:
#input
def extract_features(data): #extracting mordred features for cell_proliferation data
  calc = Calculator(descriptors)
  mols = [Chem.MolFromSmiles(smi) for smi in data["smiles"]]  
  df = calc.pandas(mols)  ## All features  
  df["status"] = data["status"].values   
  df.to_csv('/content/drive/MyDrive/ensemble/Data/proliferations_input_mordred.csv',index=False)
  return df

In [ ]:
feat = extract_features(data)

In [ ]:
print(len(feat))

#3. Ensemble Model

3.1 Preprocessing for ML Models

In [ ]:
def perform_column_pruning(data,th=75):
  data = data.replace(r'\s+', np.nan, regex=True)
  data[data == np.inf] = np.nan
  data = data.replace(r'^\s*$', np.nan, regex=True)  

  na_sum_series = data.isna().mean()
  org_data = data.copy()

  NAN_data = pd.DataFrame({0: na_sum_series.index, 1: na_sum_series.values})
  dropped = []
  for i in range(len(NAN_data)):
      if NAN_data.iloc[i][1] >= (th / 100):  # TODO check if sum or sum/length i.e. avg greater than threshold
          dropped.append(NAN_data.iloc[i][0])
  data = data.drop(dropped, axis=1)
  return data

def handle_missing_values(data):#imputation by mean
  data = data.replace([np.inf, -np.inf, "", " "], np.nan)
  data = data.replace(["", " "], np.nan)
  data.fillna(data.mean(), inplace=True)
  return data

def smote_imb(xtrain, ytrain): #upsampling by SMOTE technique
        print('Inside SMOTE, Original dataset shape %s' % Counter(ytrain))        
        sm = SMOTE(random_state=50)
        xtrain_new, ytrain_new = sm.fit_resample(xtrain, ytrain)        
        print('Inside SMOTE, Resampled dataset shape %s' % Counter(ytrain_new))
        return xtrain_new, ytrain_new

def boruta_fs(xtrain, xtest, ytrain): #boruta feature selector
    print("Inside BorutaFS, Before Shape Train: ", xtrain.shape)
    print("Inside BorutaFS, Before Shape Test: ", xtest.shape)
    rfc = RandomForestClassifier(n_estimators=100, n_jobs=-1)
    boruta_selector = BorutaPy(rfc, n_estimators='auto', random_state=50)
    boruta_selector.fit(xtrain, ytrain)
    xtrain_sel = boruta_selector.transform(xtrain)
    xtest_sel = boruta_selector.transform(xtest.values)
    print("Inside BorutaFS, After Shape Train: ", xtrain_sel.shape)
    print("Inside BorutaFS,  After Shape Test: ", xtest_sel.shape)    
    return xtrain_sel, xtest_sel

3.2 Model Classes

In [ ]:
class model_epigenetic: #Model1
    def __init__(self,train,test): #initialise train and test set       
        self.train = train
        self.test = test
    def extract_feature(self,data): #generate dataset        
        labels = data['status']
        smiles = data['smiles']
        featurizer = dc.feat.MolGraphConvFeaturizer()
        features = featurizer.featurize(smiles)            
        dataset = NumpyDataset(features,labels)  
        return dataset 
    def get_labels(self,pred_test): #Getting discrete labels from probability values    
        test_pred = []        
        for i in range(pred_test.shape[0]):
            if(pred_test[i][0]>pred_test[i][1]):
                test_pred.append(0)
            else:
                test_pred.append(1)
        return test_pred 
    def train_model(self):
        train = self.train 
        X = train['smiles']
        y = train['status']
        X_train = X
        y_train = y
        sampler = RandomOverSampler(sampling_strategy='minority') #Upsampling of minority class samples
        X_train = X_train.to_numpy()
        X_train = X_train.reshape((-1,1))
        X_train,y_train = sampler.fit_resample(X_train,y_train)
        data_train = pd.DataFrame(columns=['status','smiles'])
        data_train['status'] = y_train
        data_train['smiles'] = X_train                      
        train_dataset = self.extract_feature(data_train) #generating train dataset
        model = dc.models.GCNModel(n_tasks=1,mode='classification',graph_conv_layers=[64,64], predictor_hidden_feats=64,
        predictor_dropout = 0.2, Learning_rate = 0.001, batch_size = 32,model_dir='/content/drive/MyDrive/ensemble/models/model_epigenetic')
        #saves the trained model in model_dir
        model.fit(train_dataset,nb_epoch=100) #training for 100 epochs
       
    def test_model(self):
        test = self.test
        test_dataset = self.extract_feature(test) #generating test dataset
        model = dc.models.GCNModel(n_tasks=1,mode='classification',graph_conv_layers=[64,64], predictor_hidden_feats=64,
        predictor_dropout = 0.2, Learning_rate = 0.001, batch_size = 32,model_dir='/content/drive/MyDrive/ensemble/models/model_epigenetic')
        model.restore() #restore the trained model from model_dir
        probs = model.predict(test_dataset)    
        preds = self.get_labels(probs)
        return probs,preds


class model_oxidative:  #Model 2  
    def __init__(self,train,test):        
        self.train = train
        self.test = test
    def extract_feature(self,data):        
        labels = data['status']
        smiles = data['smiles']
        featurizer = dc.feat.MolGraphConvFeaturizer(use_edges=True) #use_edges=True for Attentive FP Model
        features = featurizer.featurize(smiles)            
        dataset = NumpyDataset(features,labels)  
        return dataset 
    def get_labels(self,pred_test): #Getting discrete labels from probability values    
        test_pred = []        
        for i in range(pred_test.shape[0]):
            if(pred_test[i][0]>pred_test[i][1]):
                test_pred.append(0)
            else:
                test_pred.append(1)
        return test_pred 
    def train_model(self):
        train = self.train 
        X = train['smiles']
        y = train['status']
        X_train = X
        y_train = y
        sampler = RandomOverSampler(sampling_strategy='minority')
        X_train = X_train.to_numpy()
        X_train = X_train.reshape((-1,1))
        X_train,y_train = sampler.fit_resample(X_train,y_train)
        data_train = pd.DataFrame(columns=['status','smiles'])
        data_train['status'] = y_train
        data_train['smiles'] = X_train                      
        train_dataset = self.extract_feature(data_train)
        model = dc.models.AttentiveFPModel(n_tasks=1,mode='classification',num_layers=2,graph_feat_size=200,dropout=0.5,batch_size=32, learning_rate=0.001,model_dir='/content/drive/MyDrive/ensemble/models/model_oxidative')
        model.fit(train_dataset,nb_epoch=100)
       
    def test_model(self):
        test = self.test
        test_dataset = self.extract_feature(test)
        model = dc.models.AttentiveFPModel(n_tasks=1,mode='classification',num_layers=2,graph_feat_size=200,dropout=0.5,batch_size=32, learning_rate=0.001,model_dir='/content/drive/MyDrive/ensemble/models/model_oxidative')
        model.restore()
        probs = model.predict(test_dataset)    
        preds = self.get_labels(probs)
        return probs,preds

class model_genotoxic: #Model 3
    def __init__(self,train,test):        
        self.train = train
        self.test = test
    def extract_feature(self,data):        
        labels = data['status']
        smiles = data['smiles']
        featurizer = dc.feat.ConvMolFeaturizer()
        features = featurizer.featurize(smiles)            
        dataset = NumpyDataset(features,labels)  
        return dataset 
    def get_labels(self,pred_test): #Getting discrete labels from probability values    
        test_pred = []        
        for i in range(pred_test.shape[0]):
            if(pred_test[i][0][0]>pred_test[i][0][1]):
                test_pred.append(0)
            else:
                test_pred.append(1)
        return test_pred 
    def train_model(self):
        train = self.train                             
        train_dataset = self.extract_feature(train)
        model = dc.models.GraphConvModel(mode='classification',n_tasks=1,batch_size=32,learning_rate=0.001,model_dir='/content/drive/MyDrive/ensemble/models/model_genotoxic')
        model.fit(train_dataset,nb_epoch=100)
       
    def test_model(self):
        test = self.test
        test_dataset = self.extract_feature(test)
        model = dc.models.GraphConvModel(mode='classification',n_tasks=1,batch_size=32,learning_rate=0.001,model_dir='/content/drive/MyDrive/ensemble/models/model_genotoxic')
        model.restore()
        probs = model.predict(test_dataset)    
        preds = self.get_labels(probs)
        return probs,preds

class model_electrophiles: #Model 4
    def __init__(self,train,test):        
        self.train = train
        self.test = test
    def extract_feature(self,data):        
        labels = data['status']
        smiles = data['smiles']
        featurizer = dc.feat.MolGraphConvFeaturizer(use_edges=True)
        features = featurizer.featurize(smiles)            
        dataset = NumpyDataset(features,labels)  
        return dataset 
    def get_labels(self,pred_test): #Getting discrete labels from probability values    
        test_pred = []   
        print(pred_test.shape)     
        for i in range(pred_test.shape[0]):
            if(pred_test[i][0]>pred_test[i][1]):
                test_pred.append(0)
            else:
                test_pred.append(1)
        return test_pred 
    def train_model(self):
        train = self.train         
        X = train['smiles']
        y = train['status']
        X_train = X
        y_train = y
        sampler = RandomOverSampler(sampling_strategy='minority')
        X_train = X_train.to_numpy()
        X_train = X_train.reshape((-1,1))
        X_train,y_train = sampler.fit_resample(X_train,y_train)
        data_train = pd.DataFrame(columns=['status','smiles'])
        data_train['status'] = y_train
        data_train['smiles'] = X_train                      
        train_dataset = self.extract_feature(data_train)
        model = dc.models.AttentiveFPModel(n_tasks=1,mode='classification',num_layers=2,graph_feat_size=200,dropout=0,batch_size=32, learning_rate=0.001,model_dir='/content/drive/MyDrive/ensemble/models/model_electrophiles_final')
        model.fit(train_dataset,nb_epoch=100)        

    def test_model(self):
        test = self.test
        test_dataset = self.extract_feature(test)
        model = dc.models.AttentiveFPModel(n_tasks=1,mode='classification',num_layers=2,graph_feat_size=200,dropout=0,batch_size=32, learning_rate=0.001,model_dir='/content/drive/MyDrive/ensemble/models/model_electrophiles_final')
        model.restore()
        probs = model.predict(test_dataset)    
        preds = self.get_labels(probs)
        return probs,preds

class model_apoptosis: #Model 5
    def __init__(self,data):
        self.data = data

    def preprocess(self): 
        data = self.data                   
        labels = data['status']
        data.drop(columns=["status"],inplace=True)  
        y_train = labels.iloc[:569]     
        y_test = labels.iloc[569:]
        data[data.columns] = data[data.columns].apply(pd.to_numeric,errors='coerce')        
        data = perform_column_pruning(data)
        train = data.iloc[:569,:]
        test = data.iloc[569:,:]        
        train = handle_missing_values(train).values        
        test = handle_missing_values(test)
        x_train = train
        x_test = test                      
        x_train,y_train = smote_imb(x_train,y_train)     
        x_train, x_test = boruta_fs(x_train, x_test, y_train)
        self.x_train,self.y_train,self.x_test,self.y_test = x_train,y_train,x_test,y_test
    
    def train_model(self):
        x_train,y_train = self.x_train,self.y_train        
        rf = RandomForestClassifier(n_estimators=100,min_samples_split=2,min_samples_leaf=1,max_features='sqrt',max_depth=100,bootstrap=False)
        rf.fit(x_train,y_train)        
        joblib.dump(rf, '/content/drive/MyDrive/ensemble/models/model_apoptosis.sav')

    def test_model(self):
        x_test,y_test = self.x_test,self.y_test
        rf = joblib.load('/content/drive/MyDrive/ensemble/models/model_apoptosis.sav')        
        pred = rf.predict(x_test)
        prob = rf.predict_proba(x_test)
        return prob,pred

class model_proliferation: #Model 6
    def __init__(self,data):
        self.data = data

    def preprocess(self): 
        data = self.data                   
        labels = data['status']
        data.drop(columns=["status"],inplace=True)  
        y_train = labels.iloc[:280]     
        y_test = labels.iloc[280:]
        data[data.columns] = data[data.columns].apply(pd.to_numeric,errors='coerce')        
        data = perform_column_pruning(data)
        train = data.iloc[:280,:]
        test = data.iloc[280:,:]        
        train = handle_missing_values(train).values        
        test = handle_missing_values(test)
        x_train = train
        x_test = test        
        x_train,y_train = smote_imb(x_train,y_train)     
        x_train, x_test = boruta_fs(x_train, x_test, y_train)
        self.x_train,self.y_train,self.x_test,self.y_test = x_train,y_train,x_test,y_test

    def train_model(self):
        x_train,y_train = self.x_train,self.y_train        
        et = ExtraTreesClassifier(max_depth=15,n_estimators=90)
        et.fit(x_train,y_train)                        
        joblib.dump(et, '/content/drive/MyDrive/ensemble/models/model_proliferations.sav')        

    def test_model(self):
        x_test,y_test = self.x_test,self.y_test
        et = joblib.load('/content/drive/MyDrive/ensemble/models/model_proliferations.sav')        
        pred = et.predict(x_test)
        prob = et.predict_proba(x_test)
        return prob,pred

3.3 Get Predictions of six models

In [ ]:
data_test = pd.read_csv('/content/drive/MyDrive/ensemble/Data/input_clean.csv')
data_test['status'] = [0]*len(data_test) 
#dataframe to store predictions of 6 models on carcinogen data   
predictions = pd.DataFrame(columns=['carcinogens','smiles','epigenetic_0','epigenetic_1','epigenetic_preds',
'oxidative_0','oxidative_1','oxidative_preds','genotoxic_0','genotoxic_1','genotoxic_preds',
'apoptosis_0','apoptosis_1','apoptosis_preds','proliferations_0','proliferations_1','proliferations_preds',
'electrophiles_0','electrophiles_1','electrophiles_preds','ensemble_1','ensemble_preds'])
predictions['carcinogens'] = data_test['name'] #input
predictions['smiles'] = data_test['smiles']
#predictions['status'] = data_test['status']
data_train = pd.read_csv('/content/drive/MyDrive/ensemble/Data/epigenetic.csv')
m1 = model_epigenetic(data_train,data_test)
# m1.train_model()  #only need to run once - model will be saved 
data_train = pd.read_csv('/content/drive/MyDrive/ensemble/Data/oxidative.csv')
m2 = model_oxidative(data_train,data_test)
# m2.train_model()
data_train = pd.read_csv('/content/drive/MyDrive/ensemble/Data/genotoxic.csv')
m3 = model_genotoxic(data_train,data_test)
# m3.train_model()
data_train = pd.read_csv('/content/drive/MyDrive/ensemble/Data/electrophiles.csv')
m4 = model_electrophiles(data_train,data_test)
# m4.train_model()
data = pd.read_csv('/content/drive/MyDrive/ensemble/Data/proliferations_input_mordred.csv') #input         
m5 = model_proliferation(data)
m5.preprocess()
m5.train_model()  
data = pd.read_csv('/content/drive/MyDrive/ensemble/Data/apoptosis_input_mordred.csv') #input        
m6 = model_apoptosis(data)
m6.preprocess()
m6.train_model()
probs,preds = m1.test_model()
predictions['epigenetic_0'] = probs[:,0]
predictions['epigenetic_1'] = probs[:,1]
predictions['epigenetic_preds'] = preds    
probs,preds = m2.test_model()
predictions['oxidative_0'] = probs[:,0]
predictions['oxidative_1'] = probs[:,1]
predictions['oxidative_preds'] = preds    
probs,preds = m3.test_model()    
predictions['genotoxic_0'] = probs[:,0,0]
predictions['genotoxic_1'] = probs[:,0,1]
predictions['genotoxic_preds'] = preds
probs,preds = m4.test_model()   
predictions['electrophiles_0'] = probs[:,0]
predictions['electrophiles_1'] = probs[:,1]
predictions['electrophiles_preds'] = preds
probs,preds = m5.test_model()
predictions['proliferations_0'] = probs[:,0]
predictions['proliferations_1'] = probs[:,1]
predictions['proliferations_preds'] = preds      
probs,preds = m6.test_model()
predictions['apoptosis_0'] = probs[:,0]
predictions['apoptosis_1'] = probs[:,1]
predictions['apoptosis_preds'] = preds       
predictions.to_csv('/content/drive/MyDrive/ensemble/result_input.csv',index=False) 

3.4 Compute model weights

In [ ]:
def get_predictions(probs,thresh):
    preds = []
    for prob in probs:
        if prob >= thresh:
            preds.append(1)    
        else:
            preds.append(0)
    return preds

In [ ]:
def compute_weights(pred,status,thresh):  #assign weights based on accuracy of models on validation set       
    m1_1 = pred['epigenetic_1']    
    m1_pred = get_predictions(m1_1,thresh)
    m2_1 = pred['oxidative_1']
    m2_pred = get_predictions(m2_1,thresh)
    m3_1 = pred['genotoxic_1']
    m3_pred = get_predictions(m3_1,thresh)
    m4_1 = pred['apoptosis_1']    
    m4_pred = get_predictions(m4_1,thresh)
    m5_1 = pred['proliferations_1']
    m5_pred = get_predictions(m5_1,thresh)
    m6_1 = pred['electrophiles_1']
    m6_pred = get_predictions(m6_1,thresh) 
    ranks = {} #model weights
    ranks['epigenetic'] = accuracy_score(status,m1_pred)
    ranks['oxidative'] = accuracy_score(status,m2_pred)
    ranks['genotoxic'] = accuracy_score(status,m3_pred)
    ranks['apoptosis'] = accuracy_score(status,m4_pred)
    ranks['proliferations'] = accuracy_score(status,m5_pred)
    ranks['electrophiles'] = accuracy_score(status,m6_pred)   
    return ranks  

3.5 Evaluate Ensemble

In [ ]:
pred_input = pd.read_csv('/content/drive/MyDrive/ensemble/result_input.csv')
#predictions = pred_input.drop('status',axis=1)
x_test = pred_input
#weights of models
weights = {'epigenetic': 0.4640625, 'oxidative': 0.459375, 'genotoxic': 0.6296875, 'apoptosis': 1.0, 'proliferations': 0.853125, 'electrophiles': 0.5625}    
m1_1 = list(x_test['epigenetic_1'])    
m2_1 = list(x_test['oxidative_1'])    
m3_1 = list(x_test['genotoxic_1'])
m4_1 = list(x_test['apoptosis_1'])
m5_1 = list(x_test['proliferations_1'])
m6_1 = list(x_test['electrophiles_1'])
print(weights)    
probs = []
sum = weights['epigenetic'] + weights['oxidative'] + weights['genotoxic'] + weights['apoptosis'] + weights['proliferations'] + weights['electrophiles']
for i in range(len(m1_1)):
    prob_ensemble = (m1_1[i]*weights['epigenetic']+m2_1[i]*weights['oxidative']+m3_1[i]*weights['genotoxic']+m4_1[i]*weights['apoptosis']+m5_1[i]*weights['proliferations']+m6_1[i]*weights['electrophiles'])/sum
    probs.append(prob_ensemble)    
probs = np.array(probs)
preds = get_predictions(probs,0.5)
pred_input['ensemble_1'] = probs
pred_input['ensemble_preds'] = preds
pred_input.to_csv('/content/drive/MyDrive/ensemble/result_input_final.csv')